# PyTorch Basics

## PyTorch

What is PyTorch?

It’s a Python-based scientific computing package targeted at two sets of audiences:

* A replacement for NumPy to use the power of GPUs
* a deep learning research platform that provides maximum flexibility and speed

### Tensors

Tensors are similar to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

PyTorch provides functions similar to numpy to create tensors:

In [1]:
import torch

c:\Users\chinm\miniconda3\envs\ml_finance\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
x = torch.empty(2, 3) # Construct a 5x3 matrix, uninitialized
print(x)
print(x.size()) # torch.Size is a tuple, so it supports all tuple operations

tensor([[4.8485e-27, 1.4027e-42, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])
torch.Size([2, 3])


In [3]:
x = torch.rand(2, 3) # Construct a randomly initialized matrix
print(x)

tensor([[0.0404, 0.0793, 0.3350],
        [0.3403, 0.1045, 0.5564]])


In [4]:
x = torch.zeros(5, 3, dtype=torch.long) # Construct a matrix filled zeros and of dtype long
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [5]:
x = torch.tensor([5.5, 3]) # Construct a tensor directly from data
print(x)

tensor([5.5000, 3.0000])


In [7]:
# create a tensor based on an existing tensor. These methods will reuse properties of the input tensor, e.g. dtype, unless new values are provided by user
x = x.new_ones(2, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-1.9743,  1.5456,  0.5493],
        [ 1.6086, -0.5135, -0.5834]])


### Operations

There are multiple syntaxes for operations. In the following example, we will take a look at the addition operation.

In [8]:
y = torch.rand(2, 3)
print(x + y)

print(torch.add(x, y))

result = torch.empty(2, 3)
torch.add(x, y, out=result) # Providing an output tensor as argument
print(result)

y.add_(x) # In-place addition
print(y)

tensor([[-1.9007,  2.3200,  0.7025],
        [ 1.6093,  0.2891, -0.3095]])
tensor([[-1.9007,  2.3200,  0.7025],
        [ 1.6093,  0.2891, -0.3095]])
tensor([[-1.9007,  2.3200,  0.7025],
        [ 1.6093,  0.2891, -0.3095]])
tensor([[-1.9007,  2.3200,  0.7025],
        [ 1.6093,  0.2891, -0.3095]])


Any operation that mutates a tensor in-place is post-fixed with an \_. For example: x.copy_(y), x.t_(), will change x.

### Indexing

You can use standard numpy-like indexing

If you have a one element tensor, use .item() to get the value as a Python number

In [2]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([-0.5335])
-0.5334818959236145


**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described here <https://pytorch.org/docs/torch>.

### NumPy Bridge

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations, and changing one will change the other.


In [3]:
import numpy as np
a = torch.ones(5)
print(a)
b = np.array(a) # Converts torch tensor to numpy array
print(b)

a.add_(1)
print(a)
print(b)

tensor([1., 1., 1., 1., 1.])


RuntimeError: Numpy is not available

In [4]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a) # Converts numpy array to torch tensor
np.add(a, 1, out=a)
print(a)
print(b)

RuntimeError: Numpy is not available

### CUDA Tensor
Tensors can be moved onto any device using the .to method.

In [5]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([0.4665], device='cuda:0')
tensor([0.4665], dtype=torch.float64)


### Linear Regression example

torch.nn is a Neural Network package useful for constructing models which provides layers, activation functions, loss functions, etc. ( https://github.com/torch/nn )

In [6]:
from itertools import count

import torch
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F

Define a polynomial function

In [8]:
POLY_DEGREE = 4
W_target = torch.randn(POLY_DEGREE, 1) * 5
b_target = torch.randn(1) * 5

def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target.item()

Data loader and other utils

In [9]:
def make_features(x):
    """Builds features i.e. a matrix with columns [x, x^2, x^3, x^4]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** i for i in range(1, POLY_DEGREE+1)], 1)

def get_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(batch_size)
    X = make_features(random)
    y = f(X)
    return X, y

def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = 'y = '
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, len(W) - i)
    result += '{:+.2f}'.format(b[0])
    return result

Defining the model

In [10]:
# Define model
model = nn.Linear(W_target.size(0), 1)

Training loop

In [11]:
for batch_idx in count(1):
    # Get data
    batch_x, batch_y = get_batch()

    # Reset gradients
    model.zero_grad()

    # Forward pass
    output = F.smooth_l1_loss(model(batch_x), batch_y)
    loss = output.item()

    # Backward pass
    output.backward()
    
    # Apply gradients
    for param in model.parameters():
        param.data.add_(-0.1 * param.grad.data)

    # Stop criterion
    if loss < 1e-3:
        break

print('Loss: {:.6f} after {} batches'.format(loss, batch_idx))
print('==> Learned function:\t' + poly_desc(model.weight.view(-1), model.bias))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

Loss: 0.000800 after 455 batches
==> Learned function:	y = +0.96 x^4 +0.81 x^3 -9.24 x^2 -1.99 x^1 +4.56
==> Actual function:	y = +1.00 x^4 +0.86 x^3 -9.26 x^2 -2.02 x^1 +4.56


### Other Resources

Here are some other useful resources on PyTorch

* https://cs230-stanford.github.io/pytorch-getting-started.html
* https://github.com/jcjohnson/pytorch-examples
* https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html